<a href="https://colab.research.google.com/github/shenzhun/machine-learning-prep/blob/master/tensorflow/day16_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

In [16]:
mypower = layers.Lambda(lambda x: tf.math.pow(x, 2))
mypower(tf.range(5))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 0,  1,  4,  9, 16], dtype=int32)>

In [17]:
class Linear(layers.Layer):
  def __init__(self, units=32, **kwargs):
    super(Linear, self).__init__(**kwargs)
    self.units = units
  
  def build(self, input_shape):
    self.w = self.add_weight("w", 
                             shape=(input_shape[-1], self.units), 
                             initializer='random_normal', 
                             trainable=True)
    self.b = self.add_weight("b", shape=(self.units,),
                             initializer='random_normal',
                             trainable=True)
    super(Linear, self).build(input_shape)
  
  @tf.function
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b
  
  def get_config(self):
    config = super(Linear, self).get_config()
    config.update({'units': self.units})
    return config

In [18]:
linear = Linear(units=8)
print(linear.built)

linear.build(input_shape= (None, 16))
print(linear.built)

False
True


In [25]:
linear = Linear(units=8)
print(linear.built)

linear.build(input_shape=(None, 16))
print(linear.compute_output_shape(input_shape=(None, 16)))

False
(None, 8)


In [26]:
linear = Linear(units=16)
print(linear.built)

linear(tf.random.uniform((100, 64)))
print(linear.built)
config = linear.get_config()
print(config)

False
True
{'name': 'linear_2', 'trainable': True, 'dtype': 'float32', 'units': 16}


In [21]:
tf.keras.backend.clear_session()
model = models.Sequential()
model.add(Linear(units=1, input_shape=(2,)))
print("model.input_shape: ", model.input_shape)
print("model.output_shape: ", model.output_shape)
model.summary()

model.input_shape:  (None, 2)
model.output_shape:  (None, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear (Linear)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
print(model.predict(tf.constant([[3.0, 2.0], [4.0, 5.0]])))

model.save("../data/linear_model.h5", save_format="h5")
model_loaded_keras = tf.keras.models.load_model(
    "../data/linear_model.h5", custom_objects={"Linear": Linear}
)
print(model_loaded_keras.predict(tf.constant([[3.0, 2.0], [4.0, 5.0]])))

# save as tf formatted model
model.save("../data/linear_model", save_format="tf")
model_loaded_tf = tf.keras.models.load_model("../data/linear_model")
print(model_loaded_tf.predict(tf.constant([[3.0, 2.0], [4.0, 5.0]])))

[[0.372566  ]
 [0.52775025]]
[[0.372566  ]
 [0.52775025]]
INFO:tensorflow:Assets written to: ../data/linear_model/assets
[[0.372566  ]
 [0.52775025]]
